In [10]:
import ipywidgets
from matplotlib import pyplot as plt
from termcolor import colored
import cartopy.crs as ccrs
import matplotlib.colors as colors
import xarray as xr
import numpy as np
import cartopy
import matplotlib.gridspec as gridspec
from glob import iglob
from os.path import join
from functools import reduce
import pandas as pd
#import harp
import itertools
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.patches as mpatches
import imageio
from netCDF4 import Dataset
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import cartopy.io.img_tiles as cimgt
import matplotlib.pyplot as plt
import cartopy.mpl.geoaxes
import cartopy.feature as cf

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
%matplotlib inline
import numpy as np
import ipywidgets
import cartopy.crs as ccrs
from ipywidgets import Layout

In [11]:
product_path = "/home/dk/ADI/linuxdataset/INDIA2020"


#2019 data stored in a list
files2019 =sorted(list(iglob(join(product_path, "*2019*.nc"), recursive =True)))
print(colored("NO2 2019 files:","red"), len(files2019))

#2020 data stored in a different list
files2020 = sorted(list(iglob(join(product_path, "*2020*.nc"), recursive =True)))
print(colored("NO2 2020 files:","red"), len(files2020))


#sample file for testing on a 1stMARCH2019 data
file1 = files2020[0]
file2 = files2020[1]
print(colored("Product selected for analyis:","blue"), file1)
print(colored("Product selected for analyis:","blue"), file2)

NO2 2019 files: 0
NO2 2020 files: 74
Product selected for analyis: /home/dk/ADI/linuxdataset/INDIA2020/S5P_OFFL_L2__NO2____20200301T053638_20200301T071808_12341_01_010302_20200304T104828.SUB.nc
Product selected for analyis: /home/dk/ADI/linuxdataset/INDIA2020/S5P_OFFL_L2__NO2____20200301T071808_20200301T085939_12342_01_010302_20200304T121216.SUB.nc


In [12]:
#opening product ----Global Attributes
with xr.open_dataset(file1) as file1GA:
    print(colored("Global attributes:\n", "blue"), file1GA)


#opeining product METADATA
with xr.open_dataset(file1, group = "METADATA") as file1MT:
    print(colored("\nMETADATA:\n", "blue"), file1MT)
    
with xr.open_dataset(file1, group ="PRODUCT") as file1PRD:
    print(colored("\nPRODUCT GROUP:\n", "blue"), file1PRD)

Global attributes:
 <xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*
Attributes:
    Conventions:                        CF-1.7
    institution:                        KNMI
    source:                             Sentinel 5 precursor, TROPOMI, space-...
    summary:                            TROPOMI/S5P NO2 1-Orbit L2 Swath 7x3.5km
    tracking_id:                        ee1234ed-0776-44df-99fe-7acb9cc44b26
    id:                                 S5P_OFFL_L2__NO2____20200301T053638_2...
    time_reference:                     2020-03-01T00:00:00Z
    time_reference_days_since_1950:     25627
    time_reference_julian_day:          2458909.5
    time_reference_seconds_since_1970:  1583020800
    time_coverage_start:                2020-03-01T05:58:12Z
    time_coverage_end:                  2020-03-01T06:56:36Z
    time_coverage_duration:             PT3503.560S
    time_coverage_resolution:           PT0.840S
    orbit:                              12341
    references:   

In [13]:
#DATA VARIABLES ACCESS
no2 = file1PRD['nitrogendioxide_tropospheric_column']
print(colored("\nAttributes:", "blue"), no2.attrs)
print(colored("\nValues:", "blue"), no2.values)
print(colored('Dimension of axis:', "blue"), no2.dims)
print(colored("\nCoordinates", "blue"), no2.coords)



Attributes: {'units': 'mol m-2', 'standard_name': 'troposphere_mole_content_of_nitrogen_dioxide', 'long_name': 'Tropospheric vertical column of nitrogen dioxide', 'ancillary_variables': 'nitrogendioxide_tropospheric_column_precision air_mass_factor_troposphere air_mass_factor_total averaging_kernel', 'multiplication_factor_to_convert_to_molecules_percm2': 6.02214e+19}

Values: [[[           nan            nan            nan ...            nan
              nan            nan]
  [           nan            nan            nan ...            nan
              nan            nan]
  [           nan            nan            nan ...            nan
              nan            nan]
  ...
  [           nan -2.0235169e-05  5.5254413e-06 ...            nan
              nan            nan]
  [           nan -2.9411308e-06            nan ...            nan
              nan            nan]
  [           nan            nan            nan ...            nan
              nan            nan]]]
Dimen

In [14]:
print(no2[0])

<xarray.DataArray 'nitrogendioxide_tropospheric_column' (scanline: 727, ground_pixel: 293)>
array([[          nan,           nan,           nan, ...,           nan,
                  nan,           nan],
       [          nan,           nan,           nan, ...,           nan,
                  nan,           nan],
       [          nan,           nan,           nan, ...,           nan,
                  nan,           nan],
       ...,
       [          nan, -2.023517e-05,  5.525441e-06, ...,           nan,
                  nan,           nan],
       [          nan, -2.941131e-06,           nan, ...,           nan,
                  nan,           nan],
       [          nan,           nan,           nan, ...,           nan,
                  nan,           nan]], dtype=float32)
Coordinates:
    longitude     (scanline, ground_pixel) float32 ...
    latitude      (scanline, ground_pixel) float32 ...
    time          datetime64[ns] 2020-03-01
  * scanline      (scanline) float64 2.32

In [15]:
### Converting values to standard units
### Filtering out data
no2 = no2*(6.02214e+19)
no2_filter = no2.where(file1PRD['qa_value'] > 0.75, drop = True)

### Average Data Collection

In [16]:
no2_new = no2
out = np.nan_to_num(no2_new)  
print(out)
print(no2_new)
print(no2_new.size)
print(np.count_nonzero(no2_new))

print(np.sum(no2_new))

[[[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
    0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
    0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
    0.0000000e+00  0.0000000e+00]
  ...
  [ 0.0000000e+00 -1.2185903e+15  3.3274981e+14 ...  0.0000000e+00
    0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00 -1.7711902e+14  0.0000000e+00 ...  0.0000000e+00
    0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
    0.0000000e+00  0.0000000e+00]]]
<xarray.DataArray 'nitrogendioxide_tropospheric_column' (time: 1, scanline: 727, ground_pixel: 293)>
array([[[           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,            nan, ...,
                    nan,            nan,            nan],
        [           nan,            nan,     

In [17]:
### Obtaining data from all files
import sys
for i in files2020:
    file1PRD = xr.open_dataset(i, group ="PRODUCT")
    no2 = file1PRD['nitrogendioxide_tropospheric_column']
    no2_new = np.nan_to_num(no2.values)
    sumele = np.sum(no2_new)
    nonzero = np.count_nonzero(no2_new)
    average = sumele/nonzero
    name = i[56:64]
    print(name," ", average)
    

20200301   1.1656820423809555e-05
20200301   1.8515348928739954e-05
20200302   1.1875893753354868e-05
20200302   1.799711165110826e-05
20200302   9.355631706411255e-06
20200303   1.2808703586702319e-05
20200303   1.7064432275324207e-05
20200303   1.2209533524530485e-05
20200304   6.887551360169344e-06
20200304   1.638522234720964e-05
20200304   1.301860616775834e-05
20200305   2.4840906917233952e-06
20200305   1.574500126545051e-05
20200305   1.5235290463972787e-05
20200306   1.9859882650645018e-05
20200306   1.924126480718672e-05
20200307   1.7431180422959098e-05
20200307   1.6595638473965242e-05
20200307   8.85124125211655e-06
20200308   1.141339004281919e-05
20200308   1.5990675868309216e-05
20200308   1.0997081452587484e-05
20200309   4.589619125618025e-06
20200309   1.4534768974855968e-05
20200309   1.429310987581449e-05
20200310   3.2036501015054767e-06
20200310   1.571040894778038e-05
20200310   1.6559440305666397e-05
20200311   1.7558202554772868e-05
20200311   1.66405340851951

### Plots


In [9]:
from IPython.display import IFrame
ne = IFrame(src='https://www.naturalearthdata.com', width=1000, height=500)
display(ne)

In [10]:
#DATA VARIABLES ACCESS
no2 = file1PRD['nitrogendioxide_tropospheric_column']
print(colored("\nAttributes:", "blue"), no2.attrs)
print(colored("\nValues:", "blue"), no2.values)
print(colored('Dimension of axis:', "blue"), no2.dims)
print(colored("\nCoordinates", "blue"), no2.coords)


Attributes: {'units': 'mol m-2', 'standard_name': 'troposphere_mole_content_of_nitrogen_dioxide', 'long_name': 'Tropospheric vertical column of nitrogen dioxide', 'ancillary_variables': 'nitrogendioxide_tropospheric_column_precision air_mass_factor_troposphere air_mass_factor_total averaging_kernel', 'multiplication_factor_to_convert_to_molecules_percm2': 6.02214e+19}

Values: [[[          nan           nan           nan ... 1.3569348e-05
   1.9324296e-05 1.6868675e-05]
  [          nan           nan           nan ... 1.2228424e-05
   1.9901096e-05 1.6756720e-05]
  [          nan           nan           nan ... 1.3518250e-05
   1.5382320e-05 1.6913400e-05]
  ...
  [          nan           nan           nan ...           nan
             nan           nan]
  [          nan           nan           nan ...           nan
             nan           nan]
  [          nan           nan           nan ...           nan
             nan           nan]]]
Dimension of axis: ('time', 'scanline', '

In [ ]:
plt.figure(figsize=(17,12))
ax = plt.subplot(111, projection=ccrs.Orthographic(central_latitude=20.7041, central_longitude=77.1025))
no2[0].plot.pcolormesh(ax=ax, x ='longitude', y='latitude', add_colorbar=True, cmap='magma_r',transform=ccrs.PlateCarree(),vmin=0)
ax.add_feature(cartopy.feature.RIVERS)
ax.set_title('NO2(Delhi) 2019/03/09')
ax.coastlines('10m')
###ADDING STATES TO THE GRAPH
states_provinces = cf.NaturalEarthFeature(category='cultural',
                                                name='admin_1_states_provinces',
                                                scale='10m',
                                                facecolor='none')

ax.add_feature(cf.LAND)
ax.add_feature(cf.COASTLINE)
ax.add_feature(states_provinces, edgecolor='gray')         
ax.add_feature(cf.LAND)
ax.add_feature(cf.OCEAN)
ax.add_feature(cf.COASTLINE)
ax.add_feature(cf.BORDERS,)
ax.add_feature(cf.LAKES, alpha=0.5 )
ax.add_feature(cf.RIVERS, alpha=1)
ax.stock_img()
ax.gridlines()

#Adding Delhi to our plot

ax.plot(28.5,77.5,'bo', markersize=7, transform=ccrs.PlateCarree())
ax.text(29, 78, "Delhi")
#Saving the plot
#plt.savefig('C:/Users/resultmay/', bbox_inches='tight', dpi =300)



### Quality Filter

In [ ]:
no2_filter = no2.where(file1PRD['qa_value'] > 0.75, drop = True)

#plot original data vs quality flag
plt.figure(figsize=(30, 10))
                
#Plot masked NO2 data                
ax2 = plt.subplot(122, projection=ccrs.Orthographic(11, 42))
no2_filter[0].plot.pcolormesh(ax=ax2, x='longitude', y='latitude', add_colorbar =True,cmap='magma_r',transform=ccrs.PlateCarree(), vmin=0)
ax2.set_tile('Filtered S_5p ,Quality Flag Layer> 0.75')
ax2.add_feature(cartopy.feature.RIVERS)
ax2.coastlines('10m')
ax2.stock_img()
ax2.gridlines()
                
### Save the file
                
plt.savefig("C:/Users/resultmay/",bbox_inches='tight', dpi=300)
                

### Delhi Subset

In [ ]:
ur = (29,77.5 )
ll = (28.3,76.8 )
no2_subset = no2_filter.where((no2_filter.longitude < ur[1]) & (no2_filter.longitude > ll[1]) & (no2_filter.latitude < ur[0]), drop = True)

#plot subset
plt.figure(figsize(12,12))
ax = plt.axes(projection=ccrs.Orthographir(11, 42))

stt_prv =cf.NaturalEarthFeature(category='cultural',name='admin_1_states_provinces_lines',scale='10m',facecolor='none')

no2_subset[0].plot.pcolormesh(ax=ax, x='longitude', y='latitude', add_colorbar=True, cmaps='magma_r',transform=ccrs.PlateCarree(), vmin=0)
ax.add_feature(cartopy.feature.LAND, edgecolors='black')
ax.add_feature(stt_prc, linewidth=0.2, edgecolor='black')
ax.set_title('NO2 concentration | qa_value >0.75')

### Add POI to map
ax.text(78, 29, 'Delhi', transform=ccrs.Geodetic());

###save figure to file
plt.savefig('/LOCATION')

In [11]:
#WIDGETS LAYOUT

land = ipywidgets.Checkbox(description='land', style={'description_width': 'initial'})
ocean = ipywidgets.Checkbox(description='ocean', style={'description_width': 'initial'})
coastline = ipywidgets.Checkbox(description='coastline', style={'description_width': 'initial'})
borders = ipywidgets.Checkbox(description='borders', style={'description_width': 'initial'})
lakes = ipywidgets.Checkbox(description='lakes', style={'description_width': 'initial'})
rivers = ipywidgets.Checkbox(description='rivers', style={'description_width': 'initial'})
states = ipywidgets.Checkbox(description='states', style={'description_width': 'initial'})
plot_locations = ipywidgets.Checkbox(description='plot_locations', style={'description_width': 'initial'})

box_layout = ipywidgets.Layout(display='inline-flex',                               
                               flex_flow='row',
                               align_items='stretch',
                               border='solid',
                               width='100%')

ui = ipywidgets.HBox([land, ocean, coastline, borders, lakes, rivers,states, plot_locations], layout=box_layout)

def create_map(land,ocean,coastline,borders,lakes,rivers,states,plot_locations):
    data_crs = ccrs.PlateCarree()
    ax = plt.figure(figsize=(12, 8))
    ax = ax.add_subplot(1, 1, 1, projection=data_crs)
    #ax.set_extent([68, 97, 7, 35], ccrs.PlateCarree())  
    #STATESLAYOUTDEFINATION
    states_provinces = cf.NaturalEarthFeature(category='cultural',
                                                name='admin_1_states_provinces',
                                                scale='10m',
                                                facecolor='none')

    # make the map global rather than have it zoom in to the extents of any plotted data
    #ax.set_global()
    if land == True: ax.add_feature(cfeature.LAND)
    if ocean == True: ax.add_feature(cfeature.OCEAN)
    if coastline == True: ax.add_feature(cfeature.COASTLINE)
    if borders == True: ax.add_feature(cfeature.BORDERS)
    if lakes == True: ax.add_feature(cfeature.LAKES, alpha=0.5)
    if rivers == True: ax.add_feature(cfeature.RIVERS)
    if states == True: ax.add_feature(cfeature.LAND),ax.add_feature(cfeature.COASTLINE),ax.add_feature(states_provinces, edgecolor='gray')    
    if plot_locations == True:
        delhi_lon, delhi_lat = 77.23, 28.61
        ax.plot(delhi_lon, delhi_lat, 'o', transform=data_crs)
        ax.text(delhi_lon + 1, delhi_lat - 6, 'Delhi', horizontalalignment='left', transform=data_crs)
        
out = ipywidgets.interactive_output(create_map, {'land': land,
                                                 'ocean': ocean,
                                                 'coastline': coastline,
                                                 'borders': borders,
                                                 'lakes': lakes,
                                                 'rivers': rivers,
                                                 'states': states,
                                                 'plot_locations': plot_locations})

display(ui, out)

Output()